<center><img src="../logo.png" alt="Header" style="width: 800px;"/></center>


@Copyright (C): 2010-2020, Shenzhen Yahboom Tech  
@Author: Liusen  
@Date: 2020-02-24 11:10:02  
@LastEditors: Liusen  
@LastEditTime: 2020-02-24 11:10:02    

In [1]:
#bgr8转jpeg格式
import enum
import cv2
import numpy as np
# 线程功能操作库
import threading
import inspect
import ctypes
import traitlets
from IPython.display import display, clear_output

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

# 定义手势识别函数部分

In [9]:
import cv2 
import time
import demjson
import pygame 
from aip import AipBodyAnalysis
from aip import AipSpeech
from PIL import Image, ImageDraw, ImageFont
import numpy
import ipywidgets.widgets as widgets

# 具体手势请看官方提供 https://ai.baidu.com/ai-doc/BODY/4k3cpywrv
hand={'One':'数字1','Five':'数字5','Fist':'拳头','Ok':'OK',
      'Prayer':'祈祷','Congratulation':'作揖','Honour':'作别',
      'Heart_single':'比心心','Thumb_up':'点赞','Thumb_down':'Diss',
      'ILY':'我爱你','Palm_up':'掌心向上','Heart_1':'双手比心1',
      'Heart_2':'双手比心2','Heart_3':'双手比心3','Two':'数字2',
      'Three':'数字3','Four':'数字4','Six':'数字6','Seven':'数字7',
      'Eight':'数字8','Nine':'数字9','Rock':'Rock','Insult':'竖中指','Face':'脸'}

# 下面的key要换成自己的 
""" 人体分析 APPID AK SK """
APP_ID = ''
API_KEY = ''
SECRET_KEY = ''



#调用百度AI人体分析接口
client = AipBodyAnalysis(APP_ID, API_KEY, SECRET_KEY)

#打开摄像机并设置初始参数
g_camera = cv2.VideoCapture(0)
g_camera.set(3, 640)
g_camera.set(4, 480)
g_camera.set(5, 30)  #设置帧率
g_camera.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
g_camera.set(cv2.CAP_PROP_BRIGHTNESS, 40) #设置亮度 -64 - 64  0.0
g_camera.set(cv2.CAP_PROP_CONTRAST, 50) #设置对比度 -64 - 64  2.0
g_camera.set(cv2.CAP_PROP_EXPOSURE, 156) #设置曝光值 1.0 - 5000  156.0
ret, frame = g_camera.read()


# 定义摄像头显示组件

In [10]:
#设置摄像头显示组件
image_widget = widgets.Image(format='jpeg', width=320, height=240)  
face_widget = widgets.Image(format='jpeg', width=320, height=240)
body_widget=widgets.Image(format='jpeg', width=320, height=240)
# create a horizontal box container to place the image widget next to eachother
image_container = widgets.HBox([image_widget,face_widget,body_widget])

display(image_container)
image_widget.value = bgr8_to_jpeg(frame)
face_widget.value = bgr8_to_jpeg(frame)
body_widget.value=bgr8_to_jpeg(frame)

# 定义转换显示中文函数

In [11]:
#在图像上添加识别结果文字
def cv2ImgAddText(img, text, left, top, textColor=(0, 255, 0), textSize=20):
    if (isinstance(img, numpy.ndarray)):  # 判断是否OpenCV图片类型
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # 创建一个可以在给定图像上绘图的对象
    draw = ImageDraw.Draw(img)
    # 字体的格式
    fontStyle = ImageFont.truetype(
        "simhei.ttf", textSize, encoding="utf-8")
    # 绘制文本
    draw.text((left, top), text, textColor, font=fontStyle)
    # 转换回OpenCV格式
    return cv2.cvtColor(numpy.asarray(img), cv2.COLOR_RGB2BGR)

In [12]:
#线程相关函数
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

# 主进程

In [13]:
import base64
# body_haar = cv2.CascadeClassifier("haarcascade_upperbody.xml")
face_haar = cv2.CascadeClassifier("haarcascade_profileface.xml") 
#face_haar = cv2.CascadeClassifier("haarcascade_fullbody.xml")
#eye_haar = cv2.CascadeClassifier("haarcascade_eye.xml")  
eye_haar = cv2.CascadeClassifier("haarcascade_eye_tree_eyeglasses.xml")  
def Color_Recongnize():
    
    while(1):

        ret, frame = g_camera.read()

         # 1.调用手势识别 
        raw = str(client.gesture(image_widget.value))
        gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)        
        text = demjson.decode(raw)
        
        #2.人像分割
        res = client.bodySeg(bgr8_to_jpeg(frame))
        seg_res = client.bodySeg(bgr8_to_jpeg(frame))
        if 'labelmap' not in seg_res:
            continue
        labelmap = base64.b64decode(seg_res['labelmap'])
        nparr = np.frombuffer(labelmap, np.uint8)
        labelimg = cv2.imdecode(nparr,1)
        
        new_img = np.where(labelimg==1, 255, labelimg)
        
        result = cv2.bitwise_and(frame, new_img)
        body_widget.value = bgr8_to_jpeg(result)
        #clear_output(wait=True)# 刷新显示
        

        #根据手势识别结果输出文本信息
        try:
            res = text['result'][0]['classname']
        except:
            print('识别结果：什么也没识别到哦~' )
            # 1 dst 2 文字内容 3 坐标 4 5 字体大小 6 color 7 粗细 8 line type
    #       cv2.putText(frame, '未识别', (250,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,200), 2, cv2.LINE_AA) #只能显示英文
            img = cv2ImgAddText(frame, "未识别", 250, 30, (0, 0 , 255), 30)
        else:
            print('识别结果：' + hand[res])
            #cv2.putText(frame, hand[res], (250,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2, cv2.LINE_AA)
            img = cv2ImgAddText(frame, hand[res], 250, 30, (0, 255 , 0), 30)
            if(hand[res]=='竖中指'):
                print('请注意礼貌,不要竖中指!')
            if(hand[res]=='脸'):
                print('你真好看!')
            if(hand[res]=='比心心' or hand[res]=='我爱你' or hand[res]=='双手比心1' or hand[res]=='双手比心2' or hand[res]=='双手比心3' ):
                print('爱你呦!')

        image_widget.value = bgr8_to_jpeg(img)
        
        
        #3.人脸识别
        try:
            faces = face_haar.detectMultiScale(gray_img, 1.1, 3)
            for face_x,face_y,face_w,face_h in faces:
                cv2.rectangle(frame, (face_x, face_y), (face_x+face_w, face_y+face_h), (0,255,0), 2)
                (face_x, face_y, face_w, face_h) = faces[0]
                print(faces[0:1])
            eyes = eye_haar.detectMultiScale(gray_img, 1.1, 3)
            for eye_x,eye_y,eye_w,eye_h in eyes:
                cv2.rectangle(frame, (eye_x,eye_y), (eye_x+eye_w, eye_y+eye_h), (255,0,0), 2)
    
            face_widget.value = bgr8_to_jpeg(frame)
        except:
            pass
        
        
        time.sleep(0.1)


   # g_camera.release()
    #cv2.destroyAllWindows()

In [14]:
thread1 = threading.Thread(target=Color_Recongnize)
thread1.setDaemon(True)
thread1.start()

识别结果：脸
你真好看!
[[ 75 233 138 138]]
识别结果：脸
你真好看!
识别结果：脸
你真好看!
识别结果：脸
你真好看!
识别结果：脸
你真好看!
识别结果：脸
你真好看!
识别结果：脸
你真好看!
识别结果：脸
你真好看!
识别结果：脸
你真好看!
[[ 85 248 105 105]]
识别结果：脸
你真好看!
[[ 94 258  73  73]]
识别结果：脸
你真好看!
[[ 96 259  65  65]]
识别结果：脸
你真好看!
识别结果：脸
你真好看!
[[ 95 257  71  71]]
识别结果：脸
你真好看!
识别结果：脸
你真好看!
[[ 86 250  98  98]]
识别结果：点赞
[[ 85 252 103 103]]
识别结果：点赞
识别结果：点赞
识别结果：点赞
识别结果：脸
你真好看!
[[ 76 227 138 138]]
识别结果：脸
你真好看!
识别结果：脸
你真好看!
识别结果：脸
你真好看!
[[ 77 213 142 142]]
识别结果：脸
你真好看!
识别结果：脸
你真好看!
识别结果：脸
你真好看!


In [15]:
#结束进程，只有在结束时才需要执行此段代码
stop_thread(thread1)

In [16]:


 g_camera.release()




